# Part I. ETL Pipeline for Pre-Processing the Files

## Task: PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Task 1: create cluster and set up keyspace

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Task 2: Create tables to run the following queries. 

Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Task 2.1 Getting the artist, song title and song's length given sessionId and itemInSession
- For this query, sessionId and itemInsession are considered as the composite key because the result will be filtered by them (sessionId = 338, and itemInSession = 4), and also either sessionId or itemInSession alone can't be unique. 
- The information of artist, song and length will be the output columns, so they will be in the final Select statement. 
- For the above reasons, artist, song, length, as well as sessionId and itemInsession will be in the table


In [9]:
query1="CREATE TABLE IF NOT EXISTS session_lib"
query1=query1+"(sessionId int, itemInsession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInsession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_lib (sessionId, itemInsession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        #0=artist, 1=firstName, 2=gender, 3=iteminSession, 4=LastName, 5=length, 6=level, 7=location, 8=sessionId, 9=song, 10=userId
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query1="SELECT artist, song, length FROM session_lib WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows=session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, ', ', row.song, ', ', row.length)

Faithless ,  Music Matters (Mark Knight Dub) ,  495.30731201171875


### Task 2.2 Getting the artist, song title and user's first and last name given userId and sessionId
- For this query, userId and sessionId are considered partition key because the result will be filtered by them (userid = 10 and sessionid = 182). In addition, itemInSession is also in the Primary key because the output will be sorted by itemInSession. 
- The information of artist, song and user's first and last name will be the output columns, so they will be in the final Select statement. 
- For the above reasons, artist, song, firstName, LastName as well as userId, sessionId and itemInsession will be in the table

In [29]:
query2="CREATE TABLE IF NOT EXISTS user_session_lib"
query2=query2+"(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)


In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_lib (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        #0=artist, 1=firstName, 2=gender, 3=iteminSession, 4=LastName, 5=length, 6=level, 7=location, 8=sessionId, 9=song, 10=userId
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [35]:
#The specific field names (itemInSession) are used to retrieve data from the second table. This is also a very important point when we are accessing a very huge database
query2="SELECT itemInSession, artist, song, firstName, LastName FROM user_session_lib WHERE userId = 10 AND sessionid = 182"
try:
    rows=session.execute(query2)
except Exception as e:
    print(e)
# note in row.attribute, the attribute names are all lower case
for row in rows:
    print(row.iteminsession, ', ', row.artist, ', ', row.song, ', ', row.firstname, ', ', row.lastname)

0 ,  Down To The Bone ,  Keep On Keepin' On ,  Sylvie ,  Cruz
1 ,  Three Drives ,  Greece 2000 ,  Sylvie ,  Cruz
2 ,  Sebastien Tellier ,  Kilometer ,  Sylvie ,  Cruz
3 ,  Lonnie Gordon ,  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) ,  Sylvie ,  Cruz


### Task 2.3 Getting the user's first and last name given song
- For this query, song is considered partition key because the result will be filtered by it (song = 'All Hands Against His Own'). In addition, `userId` is also in the Primary key because song alone can't be unique. 
- The information of user's first and last name will be the output columns, so they will be in the final Select statement. 
- For the above firstName, LastName as well as song, userId will be in the table

In [23]:
query3="CREATE TABLE IF NOT EXISTS song_lib"
query3=query3+"(song text, firstName text, lastName text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)        

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ##Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_lib (song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        #0=artist, 1=firstName, 2=gender, 3=iteminSession, 4=LastName, 5=length, 6=level, 7=location, 8=sessionId, 9=song, 10=userId
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [34]:
# The specific field names (userid) are used to retrieve data from the third table. This is also a very important point when we are accessing a very huge database.
query3="SELECT userId, firstName, LastName FROM song_lib WHERE song ='All Hands Against His Own' "
try:
    rows=session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.userid, ', ', row.firstname, ', ', row.lastname)

29 ,  Jacqueline ,  Lynch
80 ,  Tegan ,  Levine
95 ,  Sara ,  Johnson


### Drop the tables before closing out the sessions

In [39]:
query = "drop table session_lib"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session_lib"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_lib"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()